In [1]:
import tensorflow as tf
import sys 
sys.path.append('F:\messy code\\afg ordered\second generation\illustration2vec')
import i2v #https://github.com/rezoo/illustration2vec
from PIL import Image


In [2]:
illust2vec = i2v.make_i2v_with_chainer(
    "F:\messy code\\afg ordered\second generation\illustration2vec\illust2vec_ver200.caffemodel")

In [3]:
import os
def get_images(dir):
  folders = os.listdir(dir)
  images = {}
  for folder in folders: 
    images[folder]  = os.listdir(dir+ '\\' +folder)
  return images

In [75]:
def one_image(path , i2v = illust2vec ):
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.asarray(img)
    feature  = i2v.extract_feature([img])
    return img , np.asarray(feature)

In [73]:
import random
def folder_to_paths(dir):
    dic = get_images(dir)
    paths = []
    for folder in dic:
        names = dic[folder]
        pths = ['{}\\{}\\{}'.format(dir,folder,name) for name in names]
        paths.extend(pths)
    random.shuffle(paths)
    return paths

In [78]:
def path_to_np(dir):
    imgs = []
    features = []
    counter = 0
    for path in paths:
        img , feature = one_image(path)
        imgs.append(img)
        features.append(feature)
        counter += 1
        print(counter)
    return np.asarray(imgs) , np.asarray(features)


In [79]:
def multithreading(dir): 
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


KeyboardInterrupt: 